In [3]:
import sys

import joblib

import click
import progressbar
import pyfaidx
import pandas as pd
from gtfparse import read_gtf

UsageError: Cell magic `%%notify` not found.


In [4]:
gtffile = "/Users/milessmith/workspace/ensembl/Homo_sapiens.GRCh38.94.gtf.gz"
fastafile = (
    "/Users/milessmith/workspace/ensembl/Homo_sapiens.GRCh38.dna.primary_assembly.fa"
)
feature_type = "gene"
outfile = "/Users/milessmith/workspace/merrycrispr/merrycrispr/data/test.fasta"
gene_names = ["PML", "SP100", "DAXX", "ATRX", "SIRT2", "ZCCHC14", "YWHAH"]
# gene_name = None
boundary = 0

In [5]:
records = read_gtf(gtffile)
if gene_name:
    records = records[records["gene_name"].isin(gene_names)]
if feature_type:
    records = records[records["feature"] == feature_type]
    records = records[
        [
            "seqname",
            "feature",
            "start",
            "end",
            "strand",
            "frame",
            "gene_name",
            f"{feature_type}_id",
        ]
    ].drop_duplicates()
else:
    records = records[
        ["seqname", "feature", "start", "end", "strand", "frame", "gene_name"]
    ].drop_duplicates()

INFO:root:Extracted GTF attributes: ['gene_id', 'gene_version', 'gene_name', 'gene_source', 'gene_biotype', 'transcript_id', 'transcript_version', 'transcript_name', 'transcript_source', 'transcript_biotype', 'tag', 'transcript_support_level', 'exon_number', 'exon_id', 'exon_version', 'protein_id', 'protein_version', 'ccds_id']


In [6]:
records.iloc[0,:]

seqname                    2
feature                 gene
start              230415942
end                230544090
strand                     +
frame                      0
gene_name              SP100
gene_id      ENSG00000067066
Name: 435351, dtype: object

In [7]:
print(f"{len(records)} total records found.")

print(
    f"Loading the sequences in {fastafile}.  "
    f"Note: if this is the first time opening this file, "
    "it may take a few moments as an index is built."
)
sequences = pyfaidx.Fasta(fastafile)
print(f"Finished loading {fastafile}")

7 total records found.
Loading the sequences in /Users/milessmith/workspace/ensembl/Homo_sapiens.GRCh38.dna.primary_assembly.fa.  Note: if this is the first time opening this file, it may take a few moments as an index is built.
Finished loading /Users/milessmith/workspace/ensembl/Homo_sapiens.GRCh38.dna.primary_assembly.fa


In [8]:
records["seq_hash"] = records.apply(lambda x: hash(tuple(x)), axis=1)

In [9]:
records.iloc[0].start, records.iloc[0].end

(230415942, 230544090)

In [10]:
sequences.get_seq("15", records.iloc[0].start, records.iloc[0].end, rc=False)

>15:230415942-230415941

In [11]:
rec1_sequence = sequences.get_seq("15", records.iloc[0].start, records.iloc[0].end)
type(rec1_sequence)

pyfaidx.Sequence

In [12]:
boundary = 100

In [13]:
records

,seqname,feature,start,end,strand,frame,gene_name,gene_id,seq_hash
435351,2,gene,230415942,230544090,+,0,SP100,ENSG00000067066,5768689499246979043
891536,6,gene,33318558,33323016,-,0,DAXX,ENSG00000204209,2049764221906376783
1158352,X,gene,77504878,77786233,-,0,ATRX,ENSG00000085224,8686888483982609846
2027769,15,gene,73994673,74047812,+,0,PML,ENSG00000140464,-1895779415027010472
2181459,16,gene,87406246,87492045,-,0,ZCCHC14,ENSG00000140948,8858870127203431252
2570565,19,gene,38878555,38899862,-,0,SIRT2,ENSG00000068903,2076860975559530353
2679693,22,gene,31944461,31957603,+,0,YWHAH,ENSG00000128245,5053097926184016736


In [112]:
def split_records(rec: pd.DataFrame, padding: int) -> pd.DataFrame:
    """

    :param rec: Pandas DataFrame corresponding to all features of interest.
    :param padding: The amount of space on either size of the feature to return.
    :return: A new Pandas DataFrame with twice as many rows as the input, with 
        each row corresponding to an upstream and downstream window of sequence
    """

    rec_pos_upstream = rec[rec["strand"] == "+"].reset_index()
    rec_pos_downstream = rec[rec["strand"] == "+"].reset_index()
    rec_neg_upstream = rec[rec["strand"] == "-"].reset_index()
    rec_neg_downstream = rec[rec["strand"] == "-"].reset_index()

    rec_pos_upstream["end"] = rec_pos_upstream["start"]
    rec_pos_upstream["start"] -= padding
    rec_pos_upstream["gene_name"] += "-upstream"
    # we cannot use an underscore because that is used later in the pyfaidx.Sequence objects' fancy name
    rec_pos_downstream["start"] = rec_pos_downstream["end"]
    rec_pos_downstream["end"] += padding
    rec_pos_downstream["gene_name"] += "-downstream"

    rec_neg_downstream["end"] = rec_neg_downstream["start"]
    rec_neg_downstream["start"] -= padding
    rec_neg_downstream["gene_name"] += "-downstream"
    # we cannot use an underscore because that is used later in the pyfaidx.Sequence objects' fancy name
    rec_neg_upstream["start"] = rec_neg_upstream["end"]
    rec_neg_upstream["end"] += padding
    rec_neg_upstream["gene_name"] += "-upstream"

    return pd.concat(
        [rec_pos_upstream, rec_pos_downstream, rec_neg_upstream, rec_neg_downstream]
    ).reset_index()

In [113]:
records2 = split_records(rec=records, padding=100)

In [114]:
records2

,level_0,index,seqname,feature,start,end,strand,frame,gene_name,gene_id,seq_hash
0,0,435351,2,gene,230415842,230415942,+,0,SP100-upstream,ENSG00000067066,5768689499246979043
1,1,2027769,15,gene,73994573,73994673,+,0,PML-upstream,ENSG00000140464,-1895779415027010472
2,2,2679693,22,gene,31944361,31944461,+,0,YWHAH-upstream,ENSG00000128245,5053097926184016736
3,0,435351,2,gene,230544090,230544190,+,0,SP100-downstream,ENSG00000067066,5768689499246979043
4,1,2027769,15,gene,74047812,74047912,+,0,PML-downstream,ENSG00000140464,-1895779415027010472
5,2,2679693,22,gene,31957603,31957703,+,0,YWHAH-downstream,ENSG00000128245,5053097926184016736
6,0,891536,6,gene,33323016,33323116,-,0,DAXX-upstream,ENSG00000204209,2049764221906376783
7,1,1158352,X,gene,77786233,77786333,-,0,ATRX-upstream,ENSG00000085224,8686888483982609846
8,2,2181459,16,gene,87492045,87492145,-,0,ZCCHC14-upstream,ENSG00000140948,8858870127203431252
9,3,2570565,19,gene,38899862,38899962,-,0,SIRT2-upstream,ENSG00000068903,2076860975559530353


In [19]:
records

,seqname,feature,start,end,strand,frame,gene_name,gene_id,seq_hash
435351,2,gene,230415942,230544090,+,0,SP100,ENSG00000067066,5768689499246979043
891536,6,gene,33318558,33323016,-,0,DAXX,ENSG00000204209,2049764221906376783
1158352,X,gene,77504878,77786233,-,0,ATRX,ENSG00000085224,8686888483982609846
2027769,15,gene,73994673,74047812,+,0,PML,ENSG00000140464,-1895779415027010472
2181459,16,gene,87406246,87492045,-,0,ZCCHC14,ENSG00000140948,8858870127203431252
2570565,19,gene,38878555,38899862,-,0,SIRT2,ENSG00000068903,2076860975559530353
2679693,22,gene,31944461,31957603,+,0,YWHAH,ENSG00000128245,5053097926184016736


In [115]:
def match_seq(rec: pd.Series, sequences: pyfaidx.Fasta) -> pyfaidx.Sequence:
    """

    :param rec:
    :param sequences:
    :return:
    """

    try:
        if rec["strand"] == "-":
            rev = True
        else:
            rev = False
        seq = pyfaidx.Sequence(
            name=f"{rec['gene_name']}_"
            f"{rec['feature']}_"
            f"{rec['strand']}_"
            f"{rec['start']}_"
            f"{rec['end']}_"
            f"{rec['seq_hash']}",
            seq=sequences.get_seq(
                name=rec["seqname"], start=rec["start"], end=rec["end"], rc=rev
            ).seq,
        )
        return seq
    except ValueError:
        print(
            f"problem with {rec['gene_name']} {rec['start']} "
            f"{rec['end']} {rec['seqname']} {rec['strand']}"
        )

In [116]:
records2.index

RangeIndex(start=0, stop=14, step=1)

In [117]:
alpha = [_ for _ in [pd.Series(records2.loc[_,:]) for _ in records2.index]][0]

In [118]:
rev = True if alpha['strand'] == "-" else False

In [119]:
seq = pyfaidx.Sequence(
            name=f"{alpha['gene_name']}_"
            f"{alpha['feature']}_"
            f"{alpha['strand']}_"
            f"{alpha['start']}_"
            f"{alpha['end']}_"
            f"{alpha['seq_hash']}",
            seq=sequences.get_seq(
                name=alpha["seqname"], start=alpha["start"], end=alpha["end"], rc=rev
            ).seq)

In [120]:
seq

>SP100-upstream_gene_+_230415842_230415942_5768689499246979043
ACATGGAGTTTCCCCCCCAACTCCCATGTAAACTATGCCCCCTCTTAGAGGGCAGAGCAGCTAAGGACAATTTTGTTTCAAGGACATTTTTTTTGGCTGTG

In [121]:
final_list = [match_seq(_, sequences) for _ in [pd.Series(records2.loc[_,:]) for _ in records2.index]]

In [122]:
final_list[0]

>SP100-upstream_gene_+_230415842_230415942_5768689499246979043
ACATGGAGTTTCCCCCCCAACTCCCATGTAAACTATGCCCCCTCTTAGAGGGCAGAGCAGCTAAGGACAATTTTGTTTCAAGGACATTTTTTTTGGCTGTG

In [145]:
match_seq(rec=rec, sequences=sequences)

>PML_gene_+_73994973_74047812
...

In [24]:
with open(outfile, 'w') as o_file:
    for entry in final_list:
        o_file.writelines(f"> {entry.fancy_name}\n{entry.seq}\n")